<a href="https://colab.research.google.com/github/JOSELUISMILLA/NET8APIDEMO/blob/main/Streaming_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cuaderno de: Ricardo Alonzo Fernandez Salguero

# Streaming en PySpark: Procesamiento Continuo de Datos en Big Data

El procesamiento de datos en tiempo real es fundamental en el mundo actual, donde la velocidad y la toma de decisiones basada en datos son esenciales. PySpark, una biblioteca de procesamiento de datos distribuida basada en Apache Spark, ofrece una solución poderosa para el procesamiento continuo de datos en grandes volúmenes.

## ¿Qué es el Streaming de PySpark?

Streaming en PySpark se refiere al procesamiento continuo y en tiempo real de datos. A diferencia del procesamiento por lotes, donde los datos se procesan en conjuntos fijos y predefinidos, el streaming permite que los datos se procesen a medida que llegan, lo que habilita aplicaciones en tiempo real, como análisis de tendencias, detección de anomalías y generación de alertas.

El streaming de PySpark se basa en el motor de procesamiento distribuido de Apache Spark, lo que lo hace escalable y adecuado para trabajar con grandes volúmenes de datos en clústeres de computadoras. PySpark proporciona una API fácil de usar para configurar, ejecutar y administrar flujos de datos en tiempo real.

## Principios Clave del Streaming en PySpark

Para comprender mejor el streaming en PySpark, es fundamental conocer algunos de sus principios clave:

### Micro-Batches:
Aunque el término "streaming" puede sugerir un flujo de datos ininterrumpido, en PySpark, los datos generalmente se procesan en micro-batches. Un micro-batch es una pequeña porción de datos que se recopila durante un intervalo de tiempo definido y luego se procesa. Esta abstracción permite la paralelización y la administración eficiente de recursos en entornos distribuidos.

### Tolerancia a Fallos:
PySpark está diseñado para ser altamente tolerante a fallos. En caso de que un nodo falle durante el procesamiento en tiempo real, los datos se pueden recuperar y reprocesar, lo que garantiza la confiabilidad y la integridad de los resultados.

### Ventanas Temporales:
El procesamiento de streaming a menudo implica el análisis de datos dentro de ventanas temporales. Las ventanas temporales permiten realizar cálculos y agregaciones en un conjunto específico de datos durante un período de tiempo definido, lo que facilita el análisis de tendencias y la generación de resultados en tiempo real.

## Configuración Básica del Streaming en PySpark

A continuación, exploraremos los pasos básicos para configurar un proceso de streaming en PySpark:

### Paso 1: Crear una Sesión Spark
La primera tarea en la configuración del streaming es crear una sesión Spark. La sesión Spark proporciona un punto de entrada para interactuar con el clúster de Spark y define configuraciones clave, como el modo de ejecución y el nombre de la aplicación.

```python
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("StreamingDemo") \
    .getOrCreate()
```

En este ejemplo, se crea una sesión Spark con el nombre "StreamingDemo" y se ejecuta en modo local para fines de desarrollo. En un entorno de producción, se especificaría la URL del clúster Spark en lugar de "local".

### Paso 2: Definir el Esquema de Datos
Para procesar datos en streaming de manera efectiva, es fundamental definir un esquema para los datos entrantes. El esquema especifica la estructura de los datos, incluidos los nombres de columna y los tipos de datos.

```python
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("sensor_id", IntegerType(), True),
    StructField("value", DoubleType(), True)
])
```

En este ejemplo, se define un esquema para datos que contienen un campo de tiempo (timestamp), un identificador de sensor (sensor_id) y un valor numérico (value). El tipo de datos de cada campo se especifica adecuadamente.

### Paso 3: Configurar el Origen de Datos
El siguiente paso es configurar el origen de datos que proporcionará los datos en tiempo real. PySpark admite varias fuentes de datos, como Kafka, sistemas de archivos, sockets y más. La elección de la fuente de datos depende de la aplicación específica.

```python
from pyspark.sql.functions import from_json
from pyspark.sql.types import StringType

source = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "sensor_data") \
    .load()

# Deserializar datos JSON
source = source.selectExpr("CAST(value AS STRING) as json")
```

En este ejemplo, se configura una fuente Kafka que escucha en el tema "sensor_data". Los datos se leen en formato JSON y se convierten en una columna llamada "json". La elección de Kafka es común para flujos de datos en tiempo real debido a su capacidad de transmisión escalable.

### Paso 4: Aplicar Transformaciones y Agregaciones
Una vez que los datos fluyen desde la fuente, es posible aplicar transformaciones y agregaciones en tiempo real utilizando las capacidades de PySpark.

```python
from pyspark.sql.functions import window, avg

result = source \
    .selectExpr("CAST(json AS struct<timestamp: string, sensor_id: int, value: double>)") \
    .withColumn("timestamp", source["timestamp"].cast(StringType())) \
    .withColumn("value", source["value"].cast(DoubleType())) \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window("timestamp", "5 minutes"),
        "sensor_id"
    ) \
    .agg(avg("value").alias("avg_value"))
```

En este ejemplo, se realiza una serie de transformaciones en los datos, incluida la conversión de tipos de datos, la definición de una ventana temporal de 5 minutos y el cálculo del valor promedio por sensor dentro de cada ventana. Estas transformaciones son solo un ejemplo, y las posibilidades son amplias según los requisitos de la aplicación.

### Paso 5: Escribir Resultados a un Destino
Una vez que se han aplicado las transformaciones y agregaciones, los resultados se pueden escribir en un destino específico para su posterior análisis o visualización.

```python
query = result \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()
```

En este ejemplo, los resultados se escriben en la consola para fines de depuración y

 pruebas. Sin embargo, PySpark admite una variedad de destinos, como sistemas de archivos, bases de datos, sistemas de mensajería y más. El proceso de streaming se inicia con `start()`, y `awaitTermination()` hace que el proceso espere hasta que se detenga manualmente o debido a un error.

## Ventanas Temporales en Streaming
Una característica poderosa del streaming en PySpark es la capacidad de definir ventanas temporales. Las ventanas temporales permiten agrupar y analizar datos dentro de un intervalo de tiempo específico. Esto es fundamental para el análisis en tiempo real y la generación de resultados basados en ventanas de tiempo. PySpark admite varias formas de definir ventanas temporales:

### Ventana Fija:
En una ventana fija, los datos se agrupan en intervalos de tiempo de igual duración. Por ejemplo, es posible definir una ventana de 5 minutos que agrupe los datos en intervalos de 5 minutos cada uno.

```python
windowed_data = source \
    .groupBy(
        window("timestamp", "5 minutes"),
        "sensor_id"
    ) \
    .agg(avg("value").alias("avg_value"))
```

En este ejemplo, los datos se agrupan en ventanas de 5 minutos.

### Ventana Deslizante:
Una ventana deslizante permite que los datos se superpongan en múltiples ventanas. Esto significa que un punto de datos puede estar presente en más de una ventana si cumple con los criterios de tiempo.

```python
windowed_data = source \
    .groupBy(
        window("timestamp", "5 minutes", "1 minute"),
        "sensor_id"
    ) \
    .agg(avg("value").alias("avg_value"))
```

En este ejemplo, se define una ventana de 5 minutos que se desliza cada 1 minuto. Esto permite un solapamiento de ventanas y un análisis más granular de los datos.

### Ventana de Agua:
Una ventana de agua define un período de tolerancia para los datos retrasados o desordenados. Esto es útil cuando los datos pueden llegar fuera de orden o con retraso.

```python
source = source \
    .withWatermark("timestamp", "10 minutes")
```

En este ejemplo, se define una ventana de agua de 10 minutos, lo que significa que se aceptarán datos que lleguen hasta 10 minutos después de su marca de tiempo.

## Ventajas del Streaming en PySpark

El streaming en PySpark ofrece una serie de ventajas clave para el procesamiento de datos en tiempo real:

### Escalabilidad:
PySpark se ejecuta en un clúster distribuido, lo que permite escalar horizontalmente para manejar grandes volúmenes de datos y cargas de trabajo de procesamiento intensivas.

### Tolerancia a Fallos:
PySpark está diseñado para ser altamente tolerante a fallos, lo que garantiza la disponibilidad y la integridad de los resultados en caso de fallas en los nodos.

### Abstracción de Micro-Batch:
El uso de micro-batches permite una administración eficiente de recursos y una mayor flexibilidad en la programación y el procesamiento de datos en tiempo real.

### Integración con Fuentes de Datos:
PySpark ofrece amplia compatibilidad con diversas fuentes de datos, como Kafka, sistemas de archivos, sockets, bases de datos y más, lo que facilita la ingestión de datos en tiempo real desde múltiples fuentes.

### Ventanas Temporales:
La capacidad de definir ventanas temporales facilita el análisis de tendencias, la detección de patrones y la generación de resultados basados en intervalos de tiempo específicos.

## Casos de Uso del Streaming en PySpark

El streaming en PySpark se utiliza en una amplia variedad de casos de uso en la actualidad. Algunos ejemplos incluyen:

### Análisis de Redes Sociales:
El streaming se utiliza para analizar y rastrear tendencias en redes sociales en tiempo real, lo que permite a las empresas tomar decisiones informadas y responder rápidamente a eventos y comentarios de los usuarios.

### Monitorización de Infraestructuras:
Las organizaciones utilizan el streaming para monitorizar la salud y el rendimiento de sus infraestructuras, generando alertas en tiempo real en caso de problemas.

### Detección de Anomalías:
El streaming se aplica en la detección de anomalías, donde se analizan datos continuos para identificar patrones inusuales o comportamientos anómalos.

### Procesamiento de Eventos IoT:
En el Internet de las cosas (IoT), el streaming se utiliza para procesar eventos generados por sensores y dispositivos conectados en tiempo real, lo que permite el monitoreo y la automatización de procesos.

### Seguridad y Vigilancia:
Las aplicaciones de seguridad y vigilancia aprovechan el streaming para analizar flujos de video y detectar eventos sospechosos o actividades inusuales.

## Desafíos del Streaming en PySpark

Si bien el streaming en PySpark ofrece muchas ventajas, también presenta algunos desafíos:

### Latencia:
El procesamiento en tiempo real a menudo implica latencia mínima para tomar decisiones rápidas. Optimizar el rendimiento y reducir la latencia puede ser un desafío en entornos de alta carga.

### Escalabilidad Correcta:
Si bien PySpark es altamente escalable, es importante diseñar y configurar el clúster correctamente para garantizar un rendimiento óptimo en escenarios de carga variable.

### Gestión de Ventanas Temporales:
Definir y gestionar ventanas temporales puede ser complejo, especialmente en aplicaciones con múltiples fuentes de datos y requisitos de análisis en tiempo real.

### Mantenimiento y Monitoreo:
Los flujos de datos en tiempo real requieren un mantenimiento y monitoreo constante para garantizar que los resultados sean precisos y que cualquier problema se aborde de manera oportuna.

# Streaming PySpark

### Parte 1: Configuración del Streaming de Datos de Personas
Esta parte del código se encarga de configurar un proceso de streaming de datos que simula la lectura y procesamiento de datos de personas desde un archivo CSV.

#### Paso 1: Instalar PySpark
```markdown
!pip install pyspark
```
Este paso instala la biblioteca PySpark, que se utiliza para el procesamiento de datos en el entorno de Spark.

#### Paso 2: Crear una sesión Spark
```python
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Streaming") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()
```
En este paso, se crea una sesión Spark que se utiliza para interactuar con Spark y realizar operaciones de procesamiento de datos.

#### Paso 3: Definir el esquema para los datos CSV
```python
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("person_ID", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("first", StringType(), True),
    StructField("last", StringType(), True),
    StructField("middle", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("fax", StringType(), True),
    StructField("title", StringType(), True)
])
```
En este paso, se define el esquema de datos para los datos CSV que se van a leer en el proceso de streaming. El esquema especifica el nombre y el tipo de datos de cada columna en el archivo CSV.

#### Paso 4: Leer datos CSV desde una URL
```python
import os

# URL de los datos CSV
csv_url = "https://raw.githubusercontent.com/lawlesst/vivo-sample-data/master/data/csv/people.csv"

# Descargar el archivo CSV
!wget {csv_url} -O people.csv

# Mover el archivo CSV a un directorio de streaming simulado
streaming_directory = "streaming"
os.makedirs(streaming_directory, exist_ok=True)
!mv people.csv {streaming_directory}
```
En este paso, se descarga un archivo CSV desde una URL y se mueve a un directorio de streaming simulado. Este paso es necesario porque Spark no puede leer directamente desde una URL con `readStream`.

#### Paso 5: Configurar el streaming
```python
people_df = spark.readStream \
    .format("csv") \
    .schema(schema) \
    .option("header", True) \
    .load(streaming_directory)
```
Aquí se configura el proceso de streaming. Se utiliza `readStream` para leer datos de forma continua desde el directorio de streaming simulado. Se especifica el formato como "csv", se aplica el esquema definido en el Paso 3 y se establece la opción "header" en True para indicar que la primera fila contiene nombres de columna.

#### Paso 6: Definir la transformación y almacenar los resultados
```python
results_directory = "results"
checkpoint_directory = "checkpoint"

query = people_df.writeStream \
    .format("json") \
    .queryName("selectTable") \
    .option("checkpointLocation", checkpoint_directory) \
    .option("path", results_directory) \
    .outputMode("append") \
    .start()

query.awaitTermination()
```
En este paso, se define la transformación que se realizará en los datos y se configura la salida de resultados. Los datos se escriben en formato JSON en el directorio de resultados especificado. El proceso de streaming se inicia con `start()`, y `awaitTermination()` hace que el proceso espere hasta que se detenga de manera manual o debido a un error.

### Parte 2: Generación de Datos Simulados de Personas en Streaming
Esta parte del código simula la generación de datos de personas en streaming para el proceso de Spark.

#### Paso 1: Instalar PySpark (repetido)
Este paso ya se ha explicado anteriormente en la Parte 1.

#### Paso 2: Crear una sesión Spark (repetido)
Este paso también se ha explicado anteriormente y se crea una nueva sesión Spark.

#### Paso 3: Definir el esquema para los datos CSV (repetido)
Este paso define el esquema de datos para los datos CSV simulados.

#### Paso 4: Simular datos en streaming
```python
import time
import pandas as pd
import os
import threading

# Función para generar datos simulados y escribirlos en un archivo
def generate_and_write_data(batch_size, batch_number, directory):
    for i in range(batch_number):
        # Generar datos simulados
        data = {
            "person_ID": [j for j in range(i * batch_size, (i + 1) * batch_size)],
            "name": ["Name_" + str(j) for j in range(i * batch_size, (i + 1) * batch_size)],
            # Añade aquí más campos según el esquema
        }
        df = pd.DataFrame(data)

        # Escribir en un archivo CSV en el directorio especificado
        file_path = os.path.join(directory, f"file_{i}.csv")
        df.to_csv(file_path, index=False)

        # Esperar antes de generar el siguiente lote
        time.sleep(1)

# Directorio para los archivos CSV simulados
streaming_directory = "streaming"
os.makedirs(streaming_directory, exist_ok=True)

# Iniciar la generación de datos en un hilo separado
thread = threading.Thread(target=generate_and_write_data, args=(10, 100, streaming_directory))
thread.start()
```
En este paso, se define una función `generate_and_write_data` que genera datos simulados y los escribe en archivos CSV en el directorio de streaming simulado. Luego, se inicia un hilo separado para ejecutar esta función y simular la generación de datos en streaming.

#### Paso 5: Configurar el streaming (repetido)
Este paso configura el proceso de streaming para leer los datos generados en el directorio de streaming simulado.

#### Paso 6: Procesar y Almacenar los Resultados
```python
results_directory = "results"
checkpoint_directory = "checkpoint"

# Generar un nombre de consulta único
import uuid
unique_query_name = "selectTable_" + str(uuid.uuid4())

os.makedirs(results_directory, exist_ok=True)
os.makedirs(checkpoint_directory, exist_ok=True)

query = people_df.writeStream \
    .format("json") \
    .queryName(unique_query_name) \
    .option("checkpointLocation", checkpoint_directory) \
    .option("path", results_directory) \
    .outputMode("append") \
    .start()

query.awaitTermination

()
```
En este paso, se configura la salida de resultados en formato JSON en un directorio específico. Se utiliza un nombre de consulta único para evitar conflictos. El proceso de streaming se inicia y espera su terminación.

### Parte 3: Generación de Datos Simulados de Temperatura en Streaming
Esta parte del código simula la generación de datos de temperatura en streaming para el proceso de Spark.

#### Paso 1: Instalar PySpark y Matplotlib (repetido)
Estos pasos ya se han explicado anteriormente.

#### Paso 2: Crear una sesión Spark (repetido)
Este paso también se ha explicado anteriormente y se crea una nueva sesión Spark.

#### Paso 3: Definir el esquema para los datos CSV (repetido)
Este paso define el esquema de datos para los datos de temperatura simulados.

#### Paso 4: Simular datos en streaming
```python
import time
import pandas as pd
import os
import threading

def generate_and_write_data(batch_size, batch_number, directory):
    for i in range(batch_number):
        data = {
            "timestamp": [pd.Timestamp.now() for _ in range(batch_size)],
            "temperature": [random.randint(50, 60) for _ in range(batch_size)]
        }
        df = pd.DataFrame(data)
        file_path = os.path.join(directory, f"file_{i}.csv")
        df.to_csv(file_path, index=False)
        time.sleep(1)

streaming_directory = "streaming"
os.makedirs(streaming_directory, exist_ok=True)

thread = threading.Thread(target=generate_and_write_data, args=(10, 100, streaming_directory))
thread.start()
```
En este paso, se define una función `generate_and_write_data` que genera datos simulados de temperatura y los escribe en archivos CSV en el directorio de streaming simulado. Luego, se inicia un hilo separado para ejecutar esta función y simular la generación de datos en streaming.

#### Paso 5: Configurar el streaming
```python
from pyspark.sql.functions import window

people_df = spark.readStream \
    .format("csv") \
    .schema(schema) \
    .option("header", False) \
    .load(streaming_directory)
```
Este paso configura el proceso de streaming para leer los datos generados en el directorio de streaming simulado. La diferencia aquí es que se utiliza `window` de PySpark para configurar una ventana de tiempo en la que se agruparán los datos.

#### Paso 6: Procesar y Almacenar los Resultados
```python
results_directory = "results"
checkpoint_directory = "checkpoint"

os.makedirs(results_directory, exist_ok=True)
os.makedirs(checkpoint_directory, exist_ok=True)

from pyspark.sql.functions import current_timestamp
import uuid

unique_query_name = "selectTable_" + str(uuid.uuid4())

query = people_df \
    .withColumn("current_timestamp", current_timestamp()) \
    .writeStream \
    .format("json") \
    .queryName(unique_query_name) \
    .option("checkpointLocation", checkpoint_directory) \
    .option("path", results_directory) \
    .outputMode("append") \
    .start()

query.awaitTermination()
```
En este paso, se configura la salida de resultados en formato JSON en un directorio específico. Se utiliza un nombre de consulta único y se agrega una columna de marca de tiempo actual a los datos. El proceso de streaming se inicia y espera su terminación.

### Parte 4: Combina Datos de Archivos CSV
Esta parte del código combina datos de múltiples archivos CSV en un DataFrame único.

```python
import os
import pandas as pd

directory_path = '/content/streaming/'

combined_df = pd.DataFrame()

for file_name in os.listdir(directory_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(directory_path, file_name)
        df = pd.read_csv(file_path)
        combined_df = combined_df.append(df, ignore_index=True)

combined_df.sort_values('timestamp', inplace=True)
```

Este código utiliza la biblioteca `pandas` para combinar los datos de múltiples archivos CSV en un DataFrame llamado `combined_df`. Luego, se ordenan los datos por la columna "timestamp". Este paso es útil cuando se desean combinar resultados de múltiples procesos de streaming en uno solo.

In [ ]:
# Paso 1: Instalar PySpark
!pip install pyspark

# Paso 2: Crear una sesión Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Streaming") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

# Paso 3: Definir el esquema para los datos CSV
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("person_ID", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("first", StringType(), True),
    StructField("last", StringType(), True),
    StructField("middle", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("fax", StringType(), True),
    StructField("title", StringType(), True)
])

# Paso 4: Leer datos CSV desde una URL
# Nota: Normalmente no puedes leer directamente desde una URL con readStream.
# Para simular el streaming, primero descargamos los datos y luego los leemos como un stream.

import os

# URL de los datos CSV
csv_url = "https://raw.githubusercontent.com/lawlesst/vivo-sample-data/master/data/csv/people.csv"

# Descargar el archivo CSV
!wget {csv_url} -O people.csv

# Mover el archivo CSV a un directorio de streaming simulado
streaming_directory = "streaming"
os.makedirs(streaming_directory, exist_ok=True)
!mv people.csv {streaming_directory}

# Paso 5: Configurar el streaming
people_df = spark.readStream \
    .format("csv") \
    .schema(schema) \
    .option("header", True) \
    .load(streaming_directory)

# Paso 6: Definir la transformación y almacenar los resultados
results_directory = "results"
checkpoint_directory = "checkpoint"

query = people_df.writeStream \
    .format("json") \
    .queryName("selectTable") \
    .option("checkpointLocation", checkpoint_directory) \
    .option("path", results_directory) \
    .outputMode("append") \
    .start()

query.awaitTermination()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=a9629650f54863e025f1cdd37f56d1d21cdd0df47880bacddad2cab4f93c8fb5
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
--2024-09-08 17:04:27--  https://raw.githubusercontent.com/lawlesst/vivo-sample-data/master/data/csv/people.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4095 (4.0K) [text/plain]
Saving to: ‘people.csv’

people.csv          100%[===================>]   4.00K  --.-KB/s    in 0s      

2024-09-08 17:04:27 (37.6 MB/s) - ‘people.csv’ saved [4095/4

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=41>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/sock

Py4JError: An error occurred while calling o45.awaitTermination

In [ ]:
# Paso 1: Instalar PySpark
!pip install pyspark

# Paso 2: Crear una sesión Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Streaming") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

# Paso 3: Definir el esquema para los datos CSV
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("person_ID", IntegerType(), True),
    StructField("name", StringType(), True),
    # Agrega aquí más campos según sea necesario
])

# Paso 4: Simular datos en streaming
import time
import pandas as pd
import os
import threading

# Función para generar datos simulados y escribirlos en un archivo
def generate_and_write_data(batch_size, batch_number, directory):
    for i in range(batch_number):
        # Generar datos simulados
        data = {
            "person_ID": [j for j in range(i * batch_size, (i + 1) * batch_size)],
            "name": ["Name_" + str(j) for j in range(i * batch_size, (i + 1) * batch_size)],
            # Añade aquí más campos según el esquema
        }
        df = pd.DataFrame(data)

        # Escribir en un archivo CSV en el directorio especificado
        file_path = os.path.join(directory, f"file_{i}.csv")
        df.to_csv(file_path, index=False)

        # Esperar antes de generar el siguiente lote
        time.sleep(1)

# Directorio para los archivos CSV simulados
streaming_directory = "streaming"
os.makedirs(streaming_directory, exist_ok=True)

# Iniciar la generación de datos en un hilo separado
thread = threading.Thread(target=generate_and_write_data, args=(10, 100, streaming_directory))
thread.start()

# Paso 5: Configurar el streaming
people_df = spark.readStream \
    .format("csv") \
    .schema(schema) \
    .option("header", False) \
    .load(streaming_directory)

# Paso 6: Procesar y Almacenar los Resultados
results_directory = "results"
checkpoint_directory = "checkpoint"

# Generar un nombre de consulta único
import uuid
unique_query_name = "selectTable_" + str(uuid.uuid4())

os.makedirs(results_directory, exist_ok=True)
os.makedirs(checkpoint_directory, exist_ok=True)

query = people_df.writeStream \
    .format("json") \
    .queryName(unique_query_name) \
    .option("checkpointLocation", checkpoint_directory) \
    .option("path", results_directory) \
    .outputMode("append") \
    .start()

query.awaitTermination()


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Paso 1: Instalar PySpark
!pip install pyspark matplotlib

# Paso 2: Crear una sesión Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Streaming") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

# Paso 3: Definir el esquema para los datos CSV
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
import random

schema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("temperature", IntegerType(), True)
])

# Paso 4: Simular datos en streaming
import time
import pandas as pd
import os
import threading

def generate_and_write_data(batch_size, batch_number, directory):
    for i in range(batch_number):
        data = {
            "timestamp": [pd.Timestamp.now() for _ in range(batch_size)],
            "temperature": [random.randint(50, 60) for _ in range(batch_size)]
        }
        df = pd.DataFrame(data)
        file_path = os.path.join(directory, f"file_{i}.csv")
        df.to_csv(file_path, index=False)
        time.sleep(1)

streaming_directory = "streaming"
os.makedirs(streaming_directory, exist_ok=True)

thread = threading.Thread(target=generate_and_write_data, args=(10, 100, streaming_directory))
thread.start()

# Paso 5: Configurar el streaming
from pyspark.sql.functions import window

people_df = spark.readStream \
    .format("csv") \
    .schema(schema) \
    .option("header", False) \
    .load(streaming_directory)

# Paso 6: Procesar y Almacenar los Resultados
results_directory = "results"
checkpoint_directory = "checkpoint"

os.makedirs(results_directory, exist_ok=True)
os.makedirs(checkpoint_directory, exist_ok=True)

from pyspark.sql.functions import current_timestamp
import uuid

# Generar un nombre de consulta único
unique_query_name = "selectTable_" + str(uuid.uuid4())

# Asegúrate de que las líneas siguientes estén correctamente indentadas
query = people_df \
    .withColumn("current_timestamp", current_timestamp()) \
    .writeStream \
    .format("json") \
    .queryName(unique_query_name) \
    .option("checkpointLocation", checkpoint_directory) \
    .option("path", results_directory) \
    .outputMode("append") \
    .start()

query.stop()

In [ ]:
!pip install pyspark matplotlib
# Importaciones necesarias
import os
import time
import threading
import pandas as pd
import random
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Paso 1: Crear una sesión Spark
spark = SparkSession.builder \
    .master("local") \
    .appName("Streaming") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

# Paso 2: Definir el esquema para los datos CSV
schema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("temperature", IntegerType(), True)
])

# Paso 3: Función para generar datos en streaming
def generate_and_write_data(batch_size, batch_number, directory):
    for i in range(batch_number):
        data = {
            "timestamp": [pd.Timestamp.now() for _ in range(batch_size)],
            "temperature": [random.randint(50, 60) for _ in range(batch_size)]
        }
        df = pd.DataFrame(data)
        file_path = os.path.join(directory, f"file_{i}.csv")
        df.to_csv(file_path, index=False)
        time.sleep(1)

streaming_directory = "streaming"
os.makedirs(streaming_directory, exist_ok=True)

# Iniciar la generación de datos en un hilo separado
thread = threading.Thread(target=generate_and_write_data, args=(10, 100, streaming_directory))
thread.start()

# Paso 4: Configurar el streaming
people_df = spark.readStream \
    .format("csv") \
    .schema(schema) \
    .option("header", False) \
    .load(streaming_directory)

# Paso 5: Procesar y Almacenar los Resultados (sin bloquear)
results_directory = "results"
checkpoint_directory = "checkpoint"

os.makedirs(results_directory, exist_ok=True)
os.makedirs(checkpoint_directory, exist_ok=True)

query = people_df \
    .writeStream \
    .format("json") \
    .option("checkpointLocation", checkpoint_directory) \
    .option("path", results_directory) \
    .outputMode("append") \
    .start()


import os
import pandas as pd

# Define the directory where the CSV files are stored
directory_path = '/content/streaming/'

# Initialize an empty DataFrame to hold all combined data
combined_df = pd.DataFrame()

# Loop through all the files in the directory
for file_name in os.listdir(directory_path):
    if file_name.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory_path, file_name)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Append the DataFrame to the combined DataFrame
        combined_df = combined_df.append(df, ignore_index=True)

# Now combined_df contains all the data from the CSV files
print(combined_df)


# Sort the dataframe by timestamp just to be sure
combined_df.sort_values('timestamp', inplace=True)

# Asegúrate de detener el streaming query cuando ya no sea necesario
query.stop()

AttributeError: 'DataFrame' object has no attribute 'append'